In [ ]:
#############################################################
print("论文比较图1, 需要运行0.主程序开始，建立多层嵌套决策树模型")
#############################################################
def getKerasResnetLabel(x,enc,saveName):
    model_name = saveName 
    model = keras.models.load_model(model_name)
    y= model.predict([x], batch_size=2560)
    nSamples = y.shape[0]
    ###需要将预测出的值，转换01整数,并转为数字式
    for i in range(y.shape[0]):
        tmp = y[i]
        index=  np.argmax(tmp)
        y[i] = [0]*y.shape[1]
        y[i,index]=1
 
    y= enc.inverse_transform(y)
    y= y.reshape(-1,nSamples)[0]
    
    
    return y


def hybridTest(x,y,dt,floorLabel,kerasLabel):
    nSamples,feturesNume  = x.shape
    yHyLabel  = np.zeros((nSamples,1))#混合模型预测标签
    dtLabel = np.zeros((nSamples,1))#决策树模型预测标签
    gini,yPredictProb= getDTSamplesInfo(x,dt)
    prdictMax = np.max(yPredictProb,axis=1)
    index1 = np.argmax(yPredictProb, axis = 1)
    index1 = index1.astype('int64')

    floorLabelTmp = np.array([floorLabel for i in range(nSamples) ])

    
    indexTmp1 = list(range(nSamples))
    
    dtLabel  = floorLabelTmp[indexTmp1,index1]
    yHyLabel = dtLabel.copy()
    
    
    
    tmp1 = np.where(gini>0.1)
    tmp2 = np.where(prdictMax<0.9)
    tmp3 = np.concatenate((tmp1[0],tmp2[0]),axis = 0)
    index2 = np.unique(tmp3)
 
    yHyLabel[index2] = kerasLabel[index2]
    switch2KerasIndex = index2
    hyCounter = nSamples-len(index2)                
                      
    


    print('混合识别中，决策树的数目和比例，hyCounter,%d,%.3f%%\n' %(hyCounter,hyCounter/nSamples*100))    
    
    tmp1 = classification_report(y,dtLabel)
    print('dt 准确率\n',tmp1)
    
    tmp1 = classification_report(y,yHyLabel)
    print('hybrid 准确率\n',tmp1)
    tmp1 = classification_report(y,kerasLabel)
    print('keras 准确率\n',tmp1)
    mat1num = confusion_matrix(y,yHyLabel)
    mat2acc = confusion_matrix(y,yHyLabel,normalize='pred')
    print('hybrid mat1num\n',mat1num)
    print('hybrid mat2acc\n',np.around(mat2acc , decimals=3))
    
    
    mat1num = confusion_matrix(y, kerasLabel)
    mat2acc = confusion_matrix(y, kerasLabel,normalize='pred')
    print('keras mat1num\n',mat1num)
    print('keras mat2acc\n',np.around(mat2acc , decimals=3))
    
    
    return dtLabel,yHyLabel,switch2KerasIndex

#############################################################
import warnings
warnings.filterwarnings('ignore')

print("多层嵌套准确率图1")

floorsLabel = [[43210],[4,3210],[3,210],[2,10],[0,1]];
                      
xFloorsTest = dict()
yFloorsTest = dict()
yOriginLabelFloorsTest = dict()

xFloorsTest[1] = x3210_4#每层原始数据x, 训练和对比用
yFloorsTest[1] = y3210_4#每层原始数据y，训练和对比用
yOriginLabelFloorsTest[1]= yl5#每层5label原始数据y，训练和对比用

dtPredictLabelTest = dict()
kerasPredictLabelTest = dict()
hybridPredictLabelTest = dict()                      
switch2KerasIndexTest = dict()

                      
for i in [1,2,3,4]:
    print("\n################分析第%d层########################" %i)
    x = xFloorsTest[i]
    y = yFloorsTest[i]
    print("y.unique",np.unique(y))
    

   
    if i == 1:
        dt = dt_Floor1
        saveName = "hybrid2_KerasSimple3_likeResnet_floor1.h5"
        kerasLabel = getKerasResnetLabel(x,enc_floor1,saveName)
    if i == 2:
        dt = dt_Floor2
        saveName = "hybrid2_KerasSimple3_likeResnet_floor2.h5"
        kerasLabel = getKerasResnetLabel(x,enc_floor2,saveName)
    if i == 3:
        dt = dt_Floor3
        saveName = "hybrid2_KerasSimple3_likeResnet_floor3.h5"
        kerasLabel = getKerasResnetLabel(x,enc_floor3,saveName)
    if i == 4:
        dt = dt_Floor4
        saveName = "hybrid2_KerasSimple3_likeResnet_floor4.h5"
        kerasLabel = getKerasResnetLabel(x,enc_floor4,saveName)
    
    
   
  
 
    floorLabel = floorsLabel[i]
    dtLabel,yHyLabel, switch2KerasIndex = hybridTest(x,y,dt,floorLabel,kerasLabel)
                      
    dtPredictLabelTest[i] = dtLabel
    kerasPredictLabelTest[i] =kerasLabel
    hybridPredictLabelTest[i] = yHyLabel               
    switch2KerasIndexTest[i] =switch2KerasIndex
    
    
    y5Label = yOriginLabelFloorsTest[i]#当前层x数据，对应的5label数据
    if i == 1:#将3210转为210,3
        indexTmp = np.where(yHyLabel == 3210)#上级预测为3210
        xFloorsTest[2] = x[indexTmp]#获得预测标签为3210的x数据
        
        tmpY= y5Label[indexTmp]#获得预测标签为3210的y数据（对应原始5label）
        yOriginLabelFloorsTest[2] = tmpY.copy()#获得预测标签为3210的y数据（对应原始5label），并保存起来给到下一层
        
        index0 = np.where( (tmpY == 2) | (tmpY == 1) | (tmpY== 0))#将其中标签为2，1，0的y数据转为标签210
        tmpY[index0] = 210
        yFloorsTest[2] = tmpY.copy()#将其中标签为2，1，0的y数据转为标签210,并给到下一层Floor
        

        
       
                      
    if i == 2:
        indexTmp = np.where(yHyLabel == 210)
        xFloorsTest[3] = x[indexTmp]#获得预测标签为210的x数据(注意x是上级预测为3210的数据)
        
        tmpY= y5Label[indexTmp]#获得预测标签为210的y数据（对应原始5label）
        yOriginLabelFloorsTest[3] = tmpY.copy()#获得预测标签为210的y数据（对应原始5label），并保存起来给到下一层
        
        index0 = np.where( (tmpY == 1) | (tmpY== 0))#将其中标签为1，0的y数据转为标签10
        tmpY[index0] = 10
        yFloorsTest[3] = tmpY.copy()#将其中标签为1，0的y数据转为标签10,并给到下一层Floor
      


    if i == 3:
        indexTmp = np.where(yHyLabel == 10)
        xFloorsTest[4] = x[indexTmp]#获得预测标签为10的x数据(注意x是上级预测为210的数据)
        
        tmpY= y5Label[indexTmp]#获得预测标签为10的y数据（对应原始5label）
        yOriginLabelFloorsTest[4] = tmpY.copy()#获得预测标签为210的y数据（对应原始5label），并保存起来给到下一层
        
        
        index0 = np.where(tmpY == 1)#将其中标签为1，0的y数据转为标签10
        tmpY[index0] = 1
        
        index0 = np.where(tmpY == 0)#将其中标签为1，0的y数据转为标签10
        tmpY[index0] = 0
        
        yFloorsTest[4] = tmpY.copy()#将其中标签为1，0的y数据转为标签10,并给到下一层Floor
     
 
    

                      
        
                      
